<a href="https://colab.research.google.com/github/LunarLynxLima/Stethoscope-AI/blob/main/Audio_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torchaudio
import torchaudio.transforms as T
import numpy as np

import wave
import contextlib

import os
import csv
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

import torch         # 2.0.1+cpu
import torchaudio    # 2.0.2+cpu
import torchaudio.functional as F
import torchaudio.transforms as T

# from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler, OneHotEncoder

import re
# import tempfile
import matplotlib.pyplot as plt
from pydub import AudioSegment

import warnings
# warnings.filterwarnings("ignore")

In [ ]:

def instance_uniform_length(waveform,sample_rate,desired_duration_seconds : int = 10,add_wav_type = 1, verbose : int  = 0):
    def clip_wav(waveform, sample_rate,desired_duration_samples):
        clipped_waveform = waveform[:, :desired_duration_samples]
        return clipped_waveform, sample_rate

    def add_wav(waveform, sample_rate,desired_duration_samples,silence = 1):
        """
        silence = 0 : no wave/silence concat;
        silence = -1 : ones concat ;
        silence = 1 : replay concat
        """

        # Calculates the amount of silence needed
        # silence_samples = desired_duration_samples - len(waveform)/sample_rate
        silence_samples = desired_duration_samples - waveform.shape[1]
        if silence == 0 :
            # null silence waveform
            silence_waveform = torch.zeros((waveform.shape[1], silence_samples))
            # Concatenate original + silence
            extended_waveform = torch.cat([waveform, silence_waveform], dim=1)
        elif silence == -1:
            # white noise silence waveform
            silence_waveform = torch.ones((waveform.shape[1], silence_samples))
            # Concatenate original + silence
            extended_waveform = torch.cat([waveform, silence_waveform], dim=1)
        elif silence == 1:
            # replay silence waveform
          rest_waveform,sample_rate = clip_wav(waveform, sample_rate,silence_samples)
          extended_waveform = torch.cat([waveform, rest_waveform], dim=1)
        else:
          return -1,-1
        return extended_waveform,sample_rate

    desired_duration_samples = int(sample_rate * desired_duration_seconds)

    # Check if the input audio is shorter than the desired duration
    if waveform.shape[1] < desired_duration_samples:
        processed_waveform,sample_rate = add_wav(waveform, sample_rate,desired_duration_samples,silence=add_wav_type)
        if verbose : print(f"type of concatenated audio is {add_wav_type}")
    else:
        processed_waveform,sample_rate = clip_wav(waveform, sample_rate,desired_duration_samples)
        if verbose : print(f"Audio Cliped")
    return processed_waveform,sample_rate

def instance_uniform_sample_rate(waveform, original_sample_rate,target_sample_rate  : int = 7000,verbose:int = 0):
    if verbose : print(f"{original_sample_rate} to {target_sample_rate}")
    if verbose:
      print(type(waveform))
      print(f"Waveform: {waveform}")
    resampled_waveform = torchaudio.transforms.Resample(original_sample_rate, target_sample_rate)(waveform)
    return resampled_waveform,target_sample_rate

def instance_uniform_amplitude(file_path : str, req_uniform_amplitude :int = -20, verbose : int = 0):
    target_amplitude = req_uniform_amplitude  # In dBFS (decibels relative to full scale)
    if file_path.endswith(".wav"):
      audio = AudioSegment.from_mp3(file_path) # AudioSegment.from_mp3 is a method in the pydub library that reads an MP3 file and returns an AudioSegment object.
      normalized_audio = audio.apply_gain(target_amplitude - audio.dBFS)
      #the apply_gain function in the pydub library is used to apply a gain (amplification) to an audio segment.
      #We can use the apply_gain method to increase or decrease the volume of the audio by a specified amount in decibels (dB)
      waveform = [normalized_audio.get_array_of_samples()]
      waveform = torch.tensor(waveform)

      sample_rate = normalized_audio.frame_rate
      return waveform, sample_rate
    return -1,-1

def normalize_wav_path(file_path: str, uniform_amplitude: bool = True, uniform_length: bool = True, req_wav_length: int = 10, add_wav_type:int=1, uniform_sample_rate: bool = True, req_sample_rate: int = 7000, verbose: int = 0):
    """
    Normalize an audio file given its path by applying optional processing steps.

    Parameters:
    - file_path: The path to the input audio file.
    - uniform_amplitude: If True, normalize the audio to a uniform amplitude (default is True).
    - uniform_length: If True, adjust the audio duration to a specified length (default is True).
    - req_wav_length: The desired duration in seconds (default is 10 seconds).
    - uniform_sample_rate: If True, set the audio sample rate to a uniform value (default is True).
    - req_sample_rate: The target sample rate (default is 7000 Hz).
    - verbose: Verbosity level (0 for no output, 1 for basic output, default is 0).

    Returns:
    - processed_waveform: The normalized audio waveform. : <class 'torch.Tensor'> == e.g. torch.Size([1, 210000])
    - processed_sample_rate: The sample rate of the normalized audio. <class 'int'>

    If the input file is empty or any processing step results in an issue, the function returns (-1, -1).

    """

    if os.path.getsize(file_path) > 0:  # Check if the input audio file is not empty

        # Load the waveform and sample rate from the audio file
        waveform, sample_rate = torchaudio.load(file_path)
        processed_waveform, processed_sample_rate = waveform, sample_rate

        # Apply optional processing steps
        if uniform_amplitude:
            # Normalize amplitude and convert the waveform to a floating-point type
            import torch
            processed_waveform, processed_sample_rate = instance_uniform_amplitude(file_path, req_uniform_amplitude = -20, verbose=verbose)
            processed_waveform = processed_waveform.to(torch.float32)

        if uniform_length:
            # Adjust audio duration to the desired length
            processed_waveform, processed_sample_rate = instance_uniform_length(processed_waveform, processed_sample_rate, desired_duration_seconds=req_wav_length, add_wav_type=add_wav_type, verbose=verbose)

        if uniform_sample_rate:
            # Set the sample rate to the desired value
            processed_waveform, processed_sample_rate = instance_uniform_sample_rate(processed_waveform, processed_sample_rate, target_sample_rate=req_sample_rate, verbose=verbose)

        return processed_waveform, processed_sample_rate

    else:
        # If the input audio file is empty, return a sentinel value
        if verbose:
            print("File is empty")
        return -1, -1

In [ ]:
# Mel-Spectogram is preferred in literature, therefore defualt

def spectrogram_feature_extraction(waveform, SAMPLE_RATE, _n_lfcc : int = 40, _n_mfcc : int = 512,type : int = 0, verbose = 0):
    # waveform, SAMPLE_RATE = torchaudio.load(instance_file_path)
    if verbose : print(f'waveform shape: {waveform.shape}; SAMPLE_RATE: {SAMPLE_RATE}')

    # Define transform
    if(type == 0) : spectrogram = T.MelSpectrogram(sample_rate=SAMPLE_RATE)  # Mel-scale Spectrogram
    if(type == 1) : spectrogram = T.Spectrogram(power=None)  # Raw Spectrogram (complex-valued)
    if(type == 2) : spectrogram = T.Spectrogram(power=2)  # Power Spectrogram (real-valued)
    if(type == 3) : spectrogram = T.LFCC(sample_rate=SAMPLE_RATE, n_lfcc=_n_lfcc)  #(n_lfcc -> parameter) Linear Frequency Cepstral Coefficient
    if(type == 4) : spectrogram = T.MFCC(sample_rate=SAMPLE_RATE, n_mfcc=_n_mfcc)  #(n_fft -> parameter) Mel Frequency Cepstral Coefficient
    if(type == 5) :
        # Compute the zero-crossing rate
        zero_crossing_rate = torchaudio.transforms.ZeroCrossingRate()(waveform)
        # Calculate the mean zero-crossing rate
        mean_zero = zero_crossing_rate.mean(dim=1)
        mean_zero = mean_zero.numpy()

        return mean_zero
    if(type == 6) :
        # Compute the Short-Time Fourier Transform (STFT)
        spectrogram = T.Spectrogram()(waveform)
        # Compute the chroma feature from the STFT
        chroma = T.Chroma()(spectrogram)
        # Calculate the mean along the time axis
        chroma_mean = chroma.mean(dim=2)
        # Convert the result to a NumPy array if needed
        chroma_mean = chroma_mean.numpy()

        return chroma
    if(type == 7) :
        # Calculate the RMS using torchaudio's rms function
        rms = torchaudio.transforms.RMS()(waveform)
        # Calculate the mean along the time axis
        rms_mean = rms.mean(dim=1)

        return rms_mean
    # Perform transform
    spec = spectrogram(waveform)
    return spec

In [ ]:
# processed_waveform, processed_sample_rate =(normalize_wav_path(r"Dataset_1\audio_and_txt_files\101_1b1_Al_sc_Meditron.wav"))
# # print(processed_waveform.shape)
# # print(processed_sample_rate)
# # print(processed_waveform)
# audio_features_extraction = spectrogram_feature_extraction(waveform=processed_waveform, SAMPLE_RATE=processed_sample_rate, _n_lfcc = 40, _n_mfcc  = 512,type = 0, verbose = 0).flatten()
# print(len(audio_features_extraction))

In [ ]:
import pandas as pd
import torchaudio
import torchaudio.transforms as T
import numpy as np
import wave
import contextlib

class AudioDataloader():
    def __init__(self, annotated_file, features_list, choice, transform=None, target_transform = None):
        dataset = pd.read_csv(annotated_file)
        self.df = pd.DataFrame(dataset)
        self.df1 = self.df[features_list]
        self.audio_files = self.df['Path']
        self.audio_labels = self.df['Diagnosis']
        self.choice = choice
        self.annotations = features_list
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        # waveform, sample_rate = torchaudio.load(self.audio_files[idx]) # torchaudio.load() returns a tensor of float type and shape of [channel, time]
        # print('Audio Tensor: ', waveform)
        # print('Size of the tensor:', waveform.size())

        normalised_waveform, normalised_sample_rate = normalize_wav_path(self.audio_files[idx], verbose = 0)
        # print(normalised_waveform.size())
        audio_features_extracted = spectrogram_feature_extraction(waveform=normalised_waveform, SAMPLE_RATE = normalised_sample_rate).flatten()
        print(audio_features_extracted.size())

        # audio_features_extracted = torch.audio_features_extracted[60000]
        # print('Audio_features_shape', audio_features_extracted.shape)
        # print('Shape: ', audio_features_extracted.size())
        # print('Normalised Waveform- ', normalised_waveform)
        # print('Normalised SR', normalised_sample_rate)
        # print()

        # clipped_waveform = waveform[:, :10] #depends on how much duration
        # normalise_waveform = torch.nn.functional.normalize(clipped_waveform, dim=1) #amplitude

        # self.normalize = T.Resample(orig_freq=44100, new_freq=16000) #frequency same normalized_waveform)

        #For time duration
        # with contextlib.closing(wave.open(self.audio_files[idx],'r')) as f:
        #   frames = f.getnframes()
        #   rate = f.getframerate()
        #   duration = frames / float(rate)
        #   print('Time Duration of the audio file: ', duration)

        # print("Sample rate of waveform: {}".format(sample_rate))
        # print()

        encodings = {'URTI':1, 'Normal':2, 'Asthma':3, 'COPD': 4, 'LRTI': 5, 'Bronchiectasis':6, 'Pneumonia':7, 'Bronchiolitis':8, 'Heart Failure + Lung Fibrosis':9,
                     'Heart Failure + COPD':10, 'Pleural Effusion':11, 'Heart Failure':12, 'Asthma + Lung Fibrosis':13, 'BRON':14, 'Lung Fibrosis':15, 'DAS':16,
                     'CAS & DAS':17, 'CAS':18}
        label = 0
        if self.target_transform == True:
          if(self.choice ==1): #binary classification
            if(self.audio_labels[idx]=='Normal'):
              label = 0
            else:
              label = 1
          else:   #multi-class classification
            label =  encodings[self.audio_labels[idx]]

        return self.annotations, audio_features_extracted, label

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split, default_collate
import sys

print('Pick the features that you want to include in the dataset: \n')
print('1. Location\n')
print('2. Acoustic Channel\n')
print('3. Equipment\n')
print('4. Diagnosis\n')
print('5. Age\n')
print('6. Gender\n')
print('7. Sound Type\n')
print('8. Chest Zone\n')

# line = [int(x) for x in input().split()]
# list_set = set(line)
# unique_feature_codes = (list(list_set))
features_names = ['Location', 'Acoustic Channel', 'Equipment', 'Diagnosis', 'Age', 'Gender', 'Sound Type', 'Chest Zone']

features = []
features.append('Patient ID')
features.append('Path')

#added
features.append('Diagnosis')
features.append('Equipment')

# for i in range(len(unique_feature_codes)):
#   features.append(features_names[unique_feature_codes[i]])

# set up DataLoader for training set
dataset = AudioDataloader(r'/content/drive/MyDrive/Stethoscope Dataset/mylist.csv',features,1, False, True)
trainset, testset = random_split(dataset, [0.7, 0.3])

loader = DataLoader(dataset, shuffle=True, batch_size=4)

Pick the features that you want to include in the dataset: 

1. Location

2. Acoustic Channel

3. Equipment

4. Diagnosis

5. Age

6. Gender

7. Sound Type

8. Chest Zone



In [ ]:
for batch in loader:
    batch_annotation, audio_batch, label_batch = batch

torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])
torch.Size([44928])


KeyboardInterrupt: ignored

In [ ]:
# create model
model = nn.Sequential(
    nn.Linear(44928, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 16),
    nn.ReLU(),
    nn.Linear(16, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# Train the model
n_epochs = 1
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()

for epoch in range(n_epochs):
    for batch in loader:
        audio_annotation, audio_batch, label_batch = batch
        # print(audio_batch.shape)
        # print('Label batch', label_batch)
        y_pred = model(audio_batch) #16 X 768  4X4

        y_pred = y_pred.reshape([4,1]).to(torch.float).flatten()
        label_batch = label_batch.reshape([4,1]).to(torch.float).flatten()
        # print('y_pred', y_pred)

        loss = loss_fn(y_pred, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# create one test tensor from the testset
annotated_data, X_test, y_test = default_collate(testset)
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
# /content/drive/MyDrive/Stethoscope Dataset/Database(1)/audio_and_txt_files/101_1b1_Al_sc_Meditron.txt

# /content/drive/MyDrive/Stethoscope Dataset/Database(2)/Audio Files/BP100_N,N,P R M,70,F.wav

# /content/drive/MyDrive/Stethoscope Dataset/Database(3)/train_wav/40069321_15.3_0_p1_981.wav

In [ ]:
dataset = AudioDataloader(r'/content/drive/MyDrive/Stethoscope Dataset/mylist.csv',features, 2, False, True)
trainset, testset = random_split(dataset, [0.7, 0.3])
loader = DataLoader(dataset, shuffle=True, batch_size=4)

In [ ]:
from sklearn.metrics import confusion_matrix
# create model
model = nn.Sequential(
    nn.Linear(768, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 16),
    nn.ReLU(),
    nn.Linear(16, 4),
    nn.ReLU(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# Train the model
n_epochs = 1
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()

for epoch in range(n_epochs):
    for batch in loader:
        audio_annotation, audio_batch, label_batch = batch

        outmap_min = torch.min(label_batch)
        outmap_max = torch.max(label_batch)
        label_batch = (label_batch - outmap_min) / (outmap_max - outmap_min)

        print(audio_batch.shape)
        print('Label batch', label_batch)
        y_pred = model(audio_batch) #16 X 768  4X4

        y_pred = y_pred.reshape([4,1]).to(torch.float).flatten()
        label_batch = label_batch.reshape([4,1]).to(torch.float).flatten()
        print('y_pred', y_pred)

        loss = loss_fn(y_pred, label_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# create one test tensor from the testset
annotated_data, X_test, y_test = default_collate(testset)
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))


In [ ]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self, num_labels, input_shape):
        super(ConvNet, self).__init()

        self.conv1 = nn.Conv1d(1, 256, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2, padding=2)
        self.conv2 = nn.Conv1d(256, 256, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool1d(kernel_size=5, stride=2, padding=2)
        self.conv3 = nn.Conv1d(256, 128, kernel_size=5, stride=1, padding=2)
        self.pool3 = nn.MaxPool1d(kernel_size=5, stride=2, padding=2)
        self.conv4 = nn.Conv1d(128, 64, kernel_size=5, stride=1, padding=2)
        self.pool4 = nn.MaxPool1d(kernel_size=5, stride=2, padding=2)
        self.conv5 = nn.Conv1d(64, 32, kernel_size=5, stride=1, padding=2)
        self.pool5 = nn.MaxPool1d(kernel_size=5, stride=2, padding=2)
        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32 * (input_shape // 32), 32)
        self.dropout2 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(32, num_labels)

        # f;

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = self.pool4(torch.relu(self.conv4(x)))
        x = self.pool5(torch.relu(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout1(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

n_epochs = 1
for epoch in range(n_epochs):
    for batch in loader:
        audio_annotation, audio_batch, label_batch = batch

        num_labels = 1  # Number of output classes
        input_shape = 768  # Size of the input data
        model = ConvNet(num_labels, input_shape)
        output = model(input_data)

        # print(audio_batch.shape)
        # print('Label batch', label_batch)
        # y_pred = model(audio_batch) #16 X 768  4X4

        # y_pred = y_pred.reshape([4,1]).to(torch.float).flatten()
        # label_batch = label_batch.reshape([4,1]).to(torch.float).flatten()
        # print('y_pred', y_pred)

        # loss = loss_fn(y_pred, label_batch)
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()